In [69]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

def load_training_set(filename):
    with open(filename, "r", encoding='utf-8') as f:
        return [[x for x in row] for row in csv.reader(f, delimiter=',')]

#data = load_training_set("movie_metadata.csv")

data = pd.read_csv('movie_metadata.csv')

str_list = []

for name, value in data.iteritems():
    if type(value[1]) == str:
         str_list.append(name)
            
num_list = data.columns.difference(str_list)
movie_num = data[num_list]
movie_num = movie_num.fillna(value=0, axis=1)
#print(movie_num.head())

ratings = movie_num['imdb_score'].values
del movie_num['imdb_score']

ratings_integers = [int(round(rating)) for rating in ratings]

X = movie_num.values
# Data Normalization
X_std = StandardScaler().fit_transform(X)

# trainingdata:
number_for_training = round(0.75*len(ratings))

movie_num_training = X_std[:number_for_training]
ratings_int_training = ratings_integers[:number_for_training]
ratings_cont_training = ratings[:number_for_training]

#testdata:

movie_num_test = X_std[number_for_training:]
ratings_int_test = ratings_integers[number_for_training:]
ratings_cont_test = ratings[number_for_training:]


# k-nearest neighbor met integers voor ratings (is niet heel accuraat):

neighbor = KNeighborsClassifier(n_neighbors = 1)
neighbor.fit(movie_num_training, ratings_int_training)
neighbor.score(movie_num_test, ratings_int_test)

# k-nearest neighbor voor continue ratings:

regressor = KNeighborsRegressor()
regressor.fit(movie_num_training, ratings_cont_training)
y_predict_train = regressor.predict(movie_num_training)
y_predict_test = regressor.predict(movie_num_test)

error = 0

for i in range(len(ratings_cont_test)):
    error += (abs(y_predict_train[i]-ratings_cont_training[i]))/ratings_cont_training[i]
training_error = (error/len(ratings_cont_training))*100

print(training_error)

error = 0

for i in range(len(ratings_cont_test)):
    error += (abs(y_predict_test[i]-ratings_cont_test[i]))/ratings_cont_test[i]
test_error = (error/len(ratings_cont_test))*100

print(test_error)

2.72843114021
17.1037964487
